# Examples of X11 window drawing in haskell

These will be a lot of example how to create windows in haskell, because I plan to write something like dzen,
but usable from the xmonad with pure haskell code without the need of creation other processes.

Source of knowledge
 * https://wiki.haskell.org/X_window_programming_in_Haskell

In [10]:
import Data.Bits
import Graphics.X11.Xlib
import Graphics.X11.Xlib.Extras
import Control.Concurrent (threadDelay)
 
helloWorld :: IO ()
helloWorld = do
 dpy <- openDisplay ""
 let dflt = defaultScreen dpy
     border = blackPixel dpy dflt
     background = whitePixel dpy dflt
 rootw <- rootWindow dpy dflt
 win <- createSimpleWindow dpy rootw 0 0 100 100 1 border background
 setTextProperty dpy win "Hello World" wM_NAME
 -- make window visible
 mapWindow dpy win
 sync dpy False
 threadDelay (10 * 100000)
 destroyWindow dpy win
 closeDisplay dpy
 

In [11]:
unmanagedWindow = do
 dpy <- openDisplay ""
 let dflt = defaultScreen dpy
     scr = defaultScreenOfDisplay dpy
 rootw <- rootWindow dpy dflt
 win <- mkUnmanagedWindow dpy scr rootw 0 0 100 100
 setTextProperty dpy win "Hello World" wM_NAME
 mapWindow dpy win
 sync dpy False
 threadDelay (10 * 100000)
 destroyWindow dpy win
 closeDisplay dpy

 
mkUnmanagedWindow :: Display
 -> Screen
 -> Window
 -> Position
 -> Position
 -> Dimension
 -> Dimension
 -> IO Window
mkUnmanagedWindow dpy scr rw x y w h = do
 let visual = defaultVisualOfScreen scr
     attrmask = cWOverrideRedirect .|. cWBorderPixel .|. cWBackPixel
 win <- allocaSetWindowAttributes $ \attributes -> do
     set_override_redirect attributes True
     set_background_pixel attributes $ whitePixel dpy (defaultScreen dpy)
     set_border_pixel attributes $ blackPixel dpy (defaultScreen dpy)
     createWindow dpy rw x y w h 1 (defaultDepthOfScreen scr) inputOutput visual attrmask attributes
 return win

In [14]:
unmanagedWindow